In [ ]:
# Band8
import mitsuba as mi
mi.variants()
mi.set_variant("scalar_spectral")
scene = mi.load_file("lego/cbox.xml")

from mitsuba import ScalarTransform4f as T

def load_sensor(r, phi, theta):
    # Apply two rotations to convert from spherical coordinates to world 3D coordinates.
    origin = T.translate([0, -1.2, 0]) @ T.rotate([0, 1, 0], phi).rotate([1, 0, 0], theta) @ mi.ScalarPoint3f([0, 0, r])

    return mi.load_dict({
        'type': 'perspective',
        'fov': 39.37,
        'to_world': T.look_at(
            origin=origin,
            target=[0, -3.7, 0],
            up=[0, 1, 0]
        ),
        'sampler': {
            'type': 'independent',
        },
        'film': {
            'type': 'specfilm',
            'width': 2000,
            'height': 2000,
            'band8': {
                'type': 'spectrum',
                'value': [(757.0, 0.01), (766.0, 0.255), (775.0, 0.51), (784.0, 0.255), (793.0, 0.01)]
            }
        },
    })

radius = 2
phis = [20.0*(i+1) for i in range(10)]
thetas = [20.0*(j+1) for j in range(10)]
# thetas = [90]

sensors = []
for phi in phis:
        for theta in thetas:
            sensors.append(load_sensor(radius,theta,phi))
sensor_count = len(sensors)

images_band8 = [mi.render(scene, spp=512, sensor=sensor) for sensor in sensors]

# Saving the rendered Images. 
import os
folder_name = "rendered_images_band8"

# Create the folder if it doesn't exist
os.makedirs(folder_name, exist_ok=True)

# Save the rendered images with descriptive filenames
for i, image in enumerate(images_band8):
    filename = f"{folder_name}/view_{i+1:04d}.png"
    mi.util.write_bitmap(filename, image)
    
print(f"Rendered images saved to: {folder_name}")